# Scraping

Obtener todos los datos a base de Scrapeo:

### Fases
- Con Selenium, obtener todas las urls de los supermercados
- Con Selenium, utilizar las URLs obtenidas, para sacar las urls de:
    - Aceite girasol
    - Aceite de oliva
    - Leche
- Con selenium, sacar las URLs de las subcategorias de:
    - Aceite de oliva
        - Suave e intenso
        - Virgen
        - Virgen extra.
    - Leche
        - Enriquecida
        - Entera, semi o desnatada
        - Sin Lactosa
    De forma que, por supermercado, nos quedaremos con:
    - URL Aceite Girasol
    - URL Aceite de Oliva Suave e Intenso
    - URL Aceite de Oliva Virgen
    - URL Aceite de Oliva Virgen extra
    - URL leche enriquecida
    - URL leche entera,semi o desnatada